In [1]:
import pandas as pd
import torch
import sklearn

In [3]:
# Read the dataset
df = pd.read_parquet("cdp-minutes-item-generation-dataset/dataset.parquet")
df

,infrastructure,session_id,event_id,transcript_path,minutes_items
0,cdp-seattle-21723dcf,001f1a4e4e3c,d916e9acd175,cdp-minutes-item-generation-dataset/transcript...,"[ADOPTION OF OTHER RESOLUTIONS, PRESENTATIONS,..."
1,cdp-seattle-21723dcf,008f4e8d253c,75bb0f7c2ba4,cdp-minutes-item-generation-dataset/transcript...,"[CB 120084, CB 120083, Call To Order, Adjournm..."
2,cdp-seattle-21723dcf,015dd602acce,30411cba563e,cdp-minutes-item-generation-dataset/transcript...,"[CB 120265, Inf 2011, Public Comment, CB 12026..."
3,cdp-seattle-21723dcf,01a6d09dd442,9f581faa5ece,cdp-minutes-item-generation-dataset/transcript...,"[Inf 1736, Inf 1735, President's Report, Appro..."
4,cdp-seattle-21723dcf,01e75165fea1,9486903291a7,cdp-minutes-item-generation-dataset/transcript...,"[Approval of the Minutes, Inf 1972, Inf 1961, ..."
...,...,...,...,...,...
516,cdp-seattle-21723dcf,fd5d2f907449,34f160b4a508,cdp-minutes-item-generation-dataset/transcript...,"[ADJOURNMENT, ADOPTION OF OTHER RESOLUTIONS, C..."
517,cdp-seattle-21723dcf,fd7f4922bcc3,d0d75a1259aa,cdp-minutes-item-generation-dataset/transcript...,"[Inf 1878, Inf 1876, Session I - 9:30 a.m., In..."
518,cdp-seattle-21723dcf,fe09e3d1564f,9fc2e3743166,cdp-minutes-item-generation-dataset/transcript...,"[Approval of the Agenda, Inf 1812, Public Comm..."
519,cdp-seattle-21723dcf,fe7c8aa0dd58,7b13838e9e3e,cdp-minutes-item-generation-dataset/transcript...,"[Preview of Today’s City Council Actions, Coun..."


In [4]:
# Read transcripts
first_row = df.iloc[0]
first_row

infrastructure                                  cdp-seattle-21723dcf
session_id                                              001f1a4e4e3c
event_id                                                d916e9acd175
transcript_path    cdp-minutes-item-generation-dataset/transcript...
minutes_items      [ADOPTION OF OTHER RESOLUTIONS, PRESENTATIONS,...
Name: 0, dtype: object

In [5]:
transcript_sentences = pd.read_csv(first_row.transcript_path)
transcript_sentences

,index,confidence,start_time,end_time,text,speaker_index,speaker_name,annotations
0,0,0.97,114.380,120.053,-- 2020 meeting of the Seattle city council wi...,0,NaN,NaN
1,1,0.97,120.053,124.791,"I'm Lorena Gonzalez, president of the Seattle ...",0,NaN,NaN
2,2,0.97,124.791,134.167,Will the clerk please call a roll?,0,NaN,NaN
3,3,0.97,134.167,135.168,Councilmember Morales?,1,NaN,NaN
4,4,0.97,135.168,137.203,Here.,2,NaN,NaN
...,...,...,...,...,...,...,...,...
1194,1194,0.97,7225.418,7230.256,You will receive additional information from m...,253,NaN,NaN
1195,1195,0.97,7230.256,7236.696,So I just wanted to give you a heads up you sh...,253,NaN,NaN
1196,1196,0.97,7236.696,7243.202,"I hope you all have a wonderful afternoon, and...",253,NaN,NaN
1197,1197,0.97,7243.202,7243.603,We are adjourned.,253,NaN,NaN


In [6]:
# Target Minutes Items
minutes_items = first_row.minutes_items
minutes_items

array(['ADOPTION OF OTHER RESOLUTIONS', 'PRESENTATIONS', 'Min 283',
       'PUBLIC COMMENT', 'IRC 257',
       'ADOPTION OF INTRODUCTION AND REFERRAL CALENDAR', 'CB 119793',
       'CB 119797', 'COMMITTEE REPORTS', 'CALL TO ORDER',
       'PAYMENT OF BILLS', 'APPROVAL OF THE AGENDA', 'ADJOURNMENT',
       'OTHER BUSINESS', 'APPROVAL OF THE JOURNAL', 'CB 119800',
       'ROLL CALL'], dtype=object)

# Model

## 1.BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension https://arxiv.org/abs/1910.13461

In [ ]:
# import the model, bart, used for text summary. This code needs large downloading. I stoped.
bart = torch.hub.load('pytorch/fairseq', 'bart.large')
bart.eval()  # disable dropout (or leave in train mode to finetune)

Apply Byte-Pair Encoding (BPE) to input text:

In [ ]:
# Here we can use a loop or a method to read in a column of text to do the ecode and decode to form the tokens
tokens = bart.encode('Hello world!')
assert tokens.tolist() == [0, 31414, 232, 328, 2]
bart.decode(tokens)  # 'Hello world!'

Extract features from BART:

In [ ]:
# Extract the last layer's features
last_layer_features = bart.extract_features(tokens)
assert last_layer_features.size() == torch.Size([1, 5, 1024])

# Extract all layer's features from decoder (layer 0 is the embedding layer)
all_layers = bart.extract_features(tokens, return_all_hiddens=True)
assert len(all_layers) == 13
assert torch.all(all_layers[-1] == last_layer_features)

Use BART for sentence-pair classification tasks:

In [ ]:
# Download BART already finetuned for MNLI
bart = torch.hub.load('pytorch/fairseq', 'bart.large.mnli')
bart.eval()  # disable dropout for evaluation

# Encode a pair of sentences and make a prediction
tokens = bart.encode('BART is a seq2seq model.', 'BART is not sequence to sequence.')
bart.predict('mnli', tokens).argmax()  # 0: contradiction

# Encode another pair of sentences
tokens = bart.encode('BART is denoising autoencoder.', 'BART is version of autoencoder.')
bart.predict('mnli', tokens).argmax()  # 2: entailment

## 2. I read through the BART paper, And I found there's another model BART improved on. And it has more direct codes.  
## The model is called Extreme Summarization, I tried use its Topic-Aware Convolutional Model for Extreme Summarization.
@InProceedings{xsum-emnlp,  
  author =      "Shashi Narayan and Shay B. Cohen and Mirella Lapata",  
  title =       "Don't Give Me the Details, Just the Summary! {T}opic-Aware Convolutional Neural Networks for Extreme Summarization",  
  booktitle =   "Proceedings of the 2018 Conference on Empirical Methods in Natural Language Processing ",  
  year =        "2018",  
  address =     "Brussels, Belgium",  
}

https://github.com/EdinburghNLP/XSum

The code I tried is in XSum-master/scripts. The dataset I have not tried how to transform to XML 
format since their model's input use XML format https://github.com/EdinburghNLP/XSum/tree/master/XSum-Dataset.